In [5]:
from IPython import get_ipython
ip = get_ipython()
ip.run_line_magic("reload_ext", "autoreload")  # these will enable module autoreloading
ip.run_line_magic("autoreload", "2")

### Load in Exports

In [6]:
import os
from glob import glob
import src.ExportProvider.IBKR.Extract as ibrk
from lxml import etree
import src.ReportingStrategies.Slovenia.ReportGeneration as sir
import src.ExportProvider.IBKR.Transform as ibrkMapper
import src.ConfigurationProvider.Configuration as conf
import src.ReportingStrategies.GenericUtilities as gu
import arrow

rootOfProject = os.getcwd()

reportsDirectory = os.path.join(rootOfProject, "imports")
generatedDirectory = os.path.join(rootOfProject, "exports")
configDirectory = os.path.join(rootOfProject, "config")

configProvider = conf.ConfigurationProvider(configDirectory)
taxPayerInfo = configProvider.getConfig()

brokerExports = glob("*.xml", root_dir=reportsDirectory)
print(brokerExports)


def loadFileAndExtractLines(file: str):
    fileLocation = os.path.join(reportsDirectory, file)
    
    with open(fileLocation) as fobj:
        xml = fobj.read()

    root = etree.fromstring(xml)
    transactions = ibrk.extractFromXML(root)
    return transactions

brokerReports = list(map(loadFileAndExtractLines, brokerExports))
mergedReports = ibrk.mergeTrades(brokerReports)

['Tax_Relevant_Export_2023.xml', 'Tax_Relevant_Export_2022.xml']


### Dividends Reports

In [7]:
reportconfig = sir.EDavkiReportConfig(fromDate=arrow.get("2023"), toDate=arrow.get("2024"), ReportType=sir.EDavkiDocumentWorkflowType.ORIGINAL)

dividendReport = sir.EDavkiDividendReport(reportConfig=reportconfig, taxPayerInfo=taxPayerInfo)
envelope = dividendReport.createReportEnvelope()

convertedCommonFormat = ibrkMapper.getGenericDividendLineFromIBRKCashTransactions(mergedReports.cashTransactions)

csvReport = dividendReport.generateDataFrameReport(convertedCommonFormat)
csvReport.to_csv(os.path.join(generatedDirectory, 'export.csv'), index=False)

xmlReport = dividendReport.generateXmlReport(convertedCommonFormat, envelope)


etree.ElementTree(xmlReport).write(os.path.join(generatedDirectory, "Doh_Div_3.xml"), xml_declaration=True, encoding='utf-8', pretty_print=True)



### Stock Trades

In [13]:
reportconfig = sir.EDavkiReportConfig(fromDate=arrow.get("2022"), toDate=arrow.get("2023"), ReportType=sir.EDavkiDocumentWorkflowType.ORIGINAL)
convertedCommonFormat = ibrkMapper.convertSegmentedTradesToGenericUnderlyingGroups(mergedReports)
gUtils = gu.GenericUtilities()


converted = gUtils.generateGenericGroupings(convertedCommonFormat)
print(converted)

# print(convertedCommonFormat)
tradeReport = sir.EDavkiTradesReport(reportConfig=reportconfig, taxPayerInfo=taxPayerInfo)
envelope = tradeReport.createReportEnvelope()


tradeCsv = tradeReport.generateDataFrameReport(converted)
tradeCsv.to_csv(os.path.join(generatedDirectory, 'export-trades.csv'), index=False)



xmlReport = tradeReport.generateXmlReport(converted, envelope)


etree.ElementTree(xmlReport).write(os.path.join(generatedDirectory, "Doh_KDVP_9.xml"), xml_declaration=True, encoding='utf-8', pretty_print=True)


Failed processing stock lot (ID: 290300028, ISIN: US6742152076), found no match
Failed processing stock lot (ID: , ISIN: MHY8900D1085), found no match
Failed processing stock lot (ID: 259223542, ISIN: FR0014008WS3), found no match
[UnderlyingGrouping(ISIN='MHY1146L1258', CountryOfOrigin=None, UnderlyingCategory=<GenericCategory.REGULAR: 'REGULAR'>, StockTrades=[TradeEventStockSold(ID='495717011', ISIN='MHY1146L1258', Ticker='CTRM', AssetClass=<GenericAssetClass.STOCK: 'STOCK'>, Date=<Arrow [2023-05-04T12:25:09-04:00]>, Quantity=-100.0, AmountPerQuantity=0.623, TotalAmount=-62.3, TaxTotal=0.0, Multiplier=1), TradeEventStockSold(ID='495717012', ISIN='MHY1146L1258', Ticker='CTRM', AssetClass=<GenericAssetClass.STOCK: 'STOCK'>, Date=<Arrow [2023-05-04T12:25:09-04:00]>, Quantity=-401.0, AmountPerQuantity=0.623, TotalAmount=-249.823, TaxTotal=0.0, Multiplier=1), TradeEventStockSold(ID='495717013', ISIN='MHY1146L1258', Ticker='CTRM', AssetClass=<GenericAssetClass.STOCK: 'STOCK'>, Date=<Arrow 

### Derivatives Report

In [10]:
reportconfig = sir.EDavkiReportConfig(fromDate=arrow.get("2023"), toDate=arrow.get("2024"), ReportType=sir.EDavkiDocumentWorkflowType.ORIGINAL)
convertedCommonFormat = ibrkMapper.getGenericDerivativeTradeLinesFromIBRKTrades(mergedReports)

# print(convertedCommonFormat)
tradeReport = sir.EDavkiDerivativeReport(reportConfig=reportconfig, taxPayerInfo=taxPayerInfo)
envelope = tradeReport.createReportEnvelope()


test = tradeReport.convertTradesToIfiItems(convertedCommonFormat)





tradeCsv = tradeReport.generateDataFrameReport(convertedCommonFormat)
tradeCsv.to_csv(os.path.join(generatedDirectory, 'export-derivatives.csv'), index=False)



xmlReport = tradeReport.generateXmlReport(convertedCommonFormat, envelope)


etree.ElementTree(xmlReport).write(os.path.join(generatedDirectory, "D_IFI_4.xml"), xml_declaration=True, encoding='utf-8', pretty_print=True)


US0378331005
